In [317]:
import pandas
import json 
import numpy as np
from scipy.stats import variation 




In [334]:
with open('../dist/coins.json') as f:
    data = json.load(f)
df = pandas.DataFrame.from_dict(data['coins'])

df = df.set_index('name')
df.algo_index = pandas.to_numeric(df['algo_index'])
df.sort_values(by='total_wattage', ascending=False)



,algo_index,algo_type,algorithm,attack_hourly_cost,attack_hourly_cost_pretty,hash_rate,hash_rate_pretty,market_cap,market_cap_order,market_cap_pretty,mining_daily_avg,mining_sd,symbol,total_wattage
name,,,,,,,,,,,,,,
Bitcoin,35,ASIC,SHA-256,285937.714353,"$285,938",44049415572850353697,"44,049 PH/s",7.134369e+10,1,$71.34 B,38.978068,0.241991,BTC,4.310550e+09
Ethereum,20,GPU,Ethash,78646.679597,"$78,647",129823385374413,130 TH/s,1.463030e+10,2,$14.63 B,6.959497,0.153543,ETH,4.983120e+08
Litecoin,0,ASIC,Scrypt,36407.467834,"$36,407",282914098349420,283 TH/s,3.758924e+09,4,$3.76 B,12.373436,0.082676,LTC,4.526626e+08
BitcoinCashABC,35,ASIC,SHA-256,13878.423862,"$13,878",2138005689756885819,"2,138 PH/s",2.998712e+09,6,$3.00 B,38.978068,0.241991,BCH,2.092191e+08
Zcash,24,GPU,Equihash,13454.184102,"$13,454",2711481134,3 GH/s,3.249105e+08,22,$324.91 M,176.380546,0.066862,ZEC,8.746713e+07
BitcoinSV,35,ASIC,SHA-256,5443.334779,"$5,443",838559251669368981,839 PH/s,1.149637e+09,12,$1.15 B,38.978068,0.241991,BSV,8.205901e+07
EthereumClassic,20,GPU,Ethash,4968.882525,"$4,969",8202217236227,8 TH/s,5.215669e+08,19,$521.57 M,6.959497,0.153543,ETC,3.148326e+07
Callisto,20,GPU,Ethash,514.094677,$514,848624654939,849 GH/s,3.755481e+06,437,$3.76 M,6.959497,0.153543,CLO,3.257347e+06
Zclassic,24,GPU,Equihash,340.466479,$340,68615713,69 MH/s,6.352207e+06,365,$6.35 M,176.380546,0.066862,ZCL,2.213410e+06


In [326]:
import requests
used_algos = [0]
algo_deviations = {}
for algo in used_algos[0:1]:
    print('Looking up algorithm #{}...'.format(algo))
    data  = requests.get('https://api.nicehash.com/stats?algo={}'.format(algo))
    print('Done')


Looking up algorithm #0...
Done


In [327]:
str_data = data.content.decode('utf-8')
str_data = str_data.replace('null([', '')
str_data = str_data.replace(']);', '')

first_split = str_data.split('],')
hash_values = [s.replace('[', '').split(',')[:2] for s in first_split]
hash_speeds = [float(b) for [a, b] in second_split]

In [328]:
from statistics import mean 

algo_deviations = {}
# Snapshots every 5 minutes, and we want to average per day
interval = 12 * 24
days_checked = 30
# Reduce it to be a year's worth of data and average it by day
hash_speeds = hash_speeds[-(interval * days_checked):]
daily_avg = [mean(hash_speeds[i * interval:(i+1)* interval]) for i in range(days_checked)]


algo_dict = {"relative_sd": variation(daily_avg),
             "mean": mean(daily_avg),
             "algo_index": algo}
key = df[df.algo_index==algo].algorithm[0]
algo_deviations[key] = algo_dict




In [330]:
algo_deviations

{'Scrypt': {'algo_index': 0,
  'mean': 11.646534722222222,
  'relative_sd': 0.022148476116595671}}